In [10]:
import torch
from torch import tensor
import pandas as pd

In [6]:
torch.manual_seed(0)
n_dim = 3
U = torch.randn(5, n_dim).round()
M = torch.randn(3, n_dim).round()


In [16]:
pd.DataFrame.to_csv?

Signature:
pd.DataFrame.to_csv(
    self,
    path_or_buf: 'FilePathOrBuffer[AnyStr] | None' = None,
    sep: 'str' = ',',
    na_rep: 'str' = '',
    float_format: 'str | None' = None,
    columns: 'Sequence[Hashable] | None' = None,
    header: 'bool_t | list[str]' = True,
    index: 'bool_t' = True,
    index_label: 'IndexLabel | None' = None,
    mode: 'str' = 'w',
    encoding: 'str | None' = None,
    compression: 'CompressionOptions' = 'infer',
    quoting: 'int | None' = None,
    quotechar: 'str' = '"',
    line_terminator: 'str | None' = None,
    chunksize: 'int | None' = None,
    date_format: 'str | None' = None,
    doublequote: 'bool_t' = True,
    escapechar: 'str | None' = None,
    decimal: 'str' = '.',
    errors: 'str' = 'strict',
    storage_options: 'StorageOptions' = None,
) -> 'str | None'
Docstring:
Write object to a comma-separated values (csv) file.

Parameters
----------
path_or_buf : str or file handle, default None
    File path or object, if None is provi

In [18]:
pd.DataFrame(M.numpy()).to_clipboard(header=False)

In [21]:
pd.Series([f'User {i+1}' for i in range(5)]).to_clipboard(index=False, header=False)

In [9]:
M

tensor([[ 0., -1.,  0.],
        [-1., -2.,  0.],
        [ 1.,  1.,  1.]])

In [22]:
pairs = [(1, 0), (0, 1), (3, 2)]
for uu, mm in pairs:
    print(f"dot(user {uu+1}, movie {mm+1}) = {U[uu] @ M[mm]}")

dot(user 2, movie 1) = 1.0
dot(user 1, movie 2) = -2.0
dot(user 4, movie 3) = -1.0
